# Введение в TensorFlow

Принципы работы с tensorflow достаточно просты. Мы должны составить граф операций, затем передать в этот граф данные и дать команду произвести вычисления.

## Реализация нейрона

### Graph mode (v1.x)

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


Для начала определим входы графа.

In [4]:
x = tf.keras.Input(shape=(2,))
y = tf.keras.Input(shape=(1,))

tf.Variable - обучаемые переменные. 

`tf.Variable(
    initial_value=None, trainable=None, validate_shape=True, caching_device=None,
    name=None, variable_def=None, dtype=None, import_scope=None, constraint=None,
    synchronization=tf.VariableSynchronization.AUTO,
    aggregation=tf.compat.v1.VariableAggregation.NONE, shape=None
)`

In [5]:
weights = tf.Variable(shape = (1, 2), initial_value=((0.1, 0.1),), dtype=tf.float32, name="w")
bias = tf.Variable(initial_value=0.0, dtype=tf.float32, name="b")

Присвоим переменной activation функцию `tf.keras.activations.sigmoid` - сигмоида.

In [13]:
activation = tf.keras.activations.sigmoid

Свяжем узлы в граф с помощью функций + и -. Для умножения матрицы на вектор используем функцию `tf.matmul`.

In [14]:
model = activation(tf.matmul(x, tf.transpose(weights)) + bias)

Любые вычисления в TensorFlow v1.x выполняются с помощью сессии. Создадим сессию с помощью команды `tf.Session()`.

In [15]:
session = tf.Session()

Перед началом работы необходимо инициализировать переменные.

In [16]:
tf.global_variables_initializer().run(session=session)

Попробуем подать модели что-нибудь на вход. Для этого воспользуемся методом `session.run`. Первый аргумент - список с графами, выходы которых необходимо вычислить, второй - словарь с парами вход графа - данные. Метод возвращает значения, полученные на выходе каждого графа из первого аргумента соответственно.

In [17]:
output = session.run([model], feed_dict={x:np.array([[1.0, 1.0]])})
print(output)

[array([[0.54983395]], dtype=float32)]


Попробуем обучить модель функции "И". 

In [18]:
x_train=np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]]).astype('float32')
y_train=np.array([0, 0, 0, 1]).astype('float32').reshape((4, 1))

Определим loss-функцию - тоже граф.

In [19]:
loss = tf.reduce_mean(tf.square(model-y))

Создадим оптимизатор и настроим его на минимизацию нашей loss-функции.

In [20]:
optimizer = tf.train.RMSPropOptimizer(0.001)
optimizer_opt = optimizer.minimize(loss)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


После создания оптимизатора придется ещё раз инициализировать переменные.

In [21]:
tf.global_variables_initializer().run(session=session)

Просто несколько раз подадим оптимизатор в `sess.run`

In [22]:
epochs = 5000
feed_dict={x:x_train, y:y_train}

for epoch in range(epochs):
    _, l = session.run([optimizer_opt, loss], feed_dict=feed_dict)
    if epoch % 100 == 0:
        print('.', end = '')

..................................................

In [23]:
output = session.run([model], feed_dict={x:np.array([[1.1, 1.1], [-0.1, -0.1]])})
print(output)

[array([[0.90961134],
       [0.0036457 ]], dtype=float32)]


После работы сессию необходимо закрыть.

In [24]:
session.close()

Для удобства можно использовать конструкцию `with`, после выхода из которой сессия закроется сама. Кроме того, больше нет необходимости передавать сессию как аргумент в некоторые функции и методы.

In [25]:
with tf.Session() as session:
    tf.global_variables_initializer().run()
    feed_dict={x:x_train, y:y_train}
    
    epochs = 5000
    for epoch in range(epochs):
        _, l = session.run([optimizer_opt, loss], feed_dict=feed_dict)
        if epoch % 100 == 0:
            print('.', end = '')
    
    output = session.run([model], feed_dict={x:np.array([[1.1, 1.1], [-0.1, -0.1]])})
    print(output)

..................................................[array([[0.90961134],
       [0.0036457 ]], dtype=float32)]


### Eager mode (v2.x)

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.1.0


tf.Variable - обучаемые переменные. 

`tf.Variable(
    initial_value=None, trainable=None, validate_shape=True, caching_device=None,
    name=None, variable_def=None, dtype=None, import_scope=None, constraint=None,
    synchronization=tf.VariableSynchronization.AUTO,
    aggregation=tf.compat.v1.VariableAggregation.NONE, shape=None
)`

In [2]:
weights = tf.Variable(shape = (1, 2), initial_value=((0.1, 0.1),), dtype=tf.float32, name="w")
bias = tf.Variable(initial_value=0.0, dtype=tf.float32, name="b")

Присвоим переменной activation функцию `tf.keras.activations.sigmoid` - сигмоида.

In [3]:
activation = tf.keras.activations.sigmoid

Свяжем узлы в граф с помощью функций + и -. Для умножения матрицы на вектор используем функцию `tf.matmul`. Для этого объявим функцию и применим [декоратор](https://pythonworld.ru/osnovy/dekoratory.html) `@tf.function`

In [4]:
@tf.function
def model(x):
    return activation(tf.matmul(x, tf.transpose(weights)) + bias)

Попробуем что-нибудь подать в такую модель.

In [5]:
output = model(tf.constant([[1.0, 1.0], [0.0, 0.0]]))
print(output)

tf.Tensor(
[[0.54983395]
 [0.5       ]], shape=(2, 1), dtype=float32)


Попробуем обучить модель функции "И". 

In [6]:
x_train=np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]]).astype('float32')
y_train=np.array([0, 0, 0, 1]).astype('float32').reshape((4, 1))

Для этого определим лосс-функцию - в v2 это должна быть тоже функция python. В данном случае можно использовать инструкцию `lambda` для создания [анонимной функции](https://pythonworld.ru/tipy-dannyx-v-python/vse-o-funkciyax-i-ix-argumentax.html) без аргументов и с возвращаемым значением, которое необходимо минимизировать.

In [7]:
loss = lambda: tf.reduce_mean(tf.square(model(x_train)-y_train))

Соберем параметры, которые необходимо обучить.

In [8]:
var_list = [weights, bias]

Также потребуется определить оптимизатор.

In [9]:
epochs = 5000
optimizer = tf.keras.optimizers.RMSprop(0.001)

Теперь можно запустить процесс обучения.

In [10]:
for epoch in range(epochs):
    optimizer.minimize(loss, var_list)
    if epoch % 100 == 0:
        print('.', end = '')

..................................................

In [11]:
output = model(tf.constant([[1.1, 1.1]]))
print(output)
output = model(tf.constant([[-0.1, -0.1]]))
print(output)

tf.Tensor([[0.91108435]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.0035015]], shape=(1, 1), dtype=float32)


## Источники

[Документация. Работа с оптимизаторами.](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Optimizer)

[Сравнение синтаксиса v1.x и v2.x](https://github.com/tensorflow/community/blob/master/rfcs/20180918-functions-not-sessions-20.md)